# [Day 13] 실전! 손가락 각도 계산하기

어제 우리는 점 3개가 있으면 각도를 구할 수 있다는 것을 배웠습니다.
오늘은 MediaPipe가 찾아준 손 랜드마크를 이용해서, 실제로 내 손가락이 몇 도나 굽혀졌는지 측정해보겠습니다.

## 1. 수학 함수 가져오기

어제 만들었던 함수들을 다시 가져와서 사용하겠습니다.
복잡해 보이지만, 그냥 "각도를 구해주는 도구"라고 생각하고 사용하면 됩니다.

In [ ]:
import math

def vector_length(v):
    return math.sqrt(v[0]**2 + v[1]**2 + v[2]**2)

def dot_product(v1, v2):
    return (v1[0]*v2[0]) + (v1[1]*v2[1]) + (v1[2]*v2[2])

def angle_between(v1, v2):
    numerator = dot_product(v1, v2)
    denominator = vector_length(v1) * vector_length(v2)
    if denominator == 0: return 0
    cos_theta = numerator / denominator
    cos_theta = max(-1.0, min(1.0, cos_theta))
    return math.degrees(math.acos(cos_theta))

## 2. 랜드마크에서 벡터 만들기

검지 손가락을 예로 들어봅시다.
- **손바닥 (0번)**
- **검지 뿌리 (5번)**
- **검지 중간 (6번)**

이 세 점이 이루는 각도를 구하려면 두 개의 벡터가 필요합니다.
1. 벡터 A: 뿌리(5) -> 손바닥(0) 방향
2. 벡터 B: 뿌리(5) -> 중간(6) 방향
*(방향은 설정하기 나름이지만, 기준을 잡아야 합니다)*

MediaPipe 랜드마크 객체는 `x, y, z` 속성을 가지고 있으므로, 이를 리스트 `[x, y, z]`로 바꿔주는 작업이 필요합니다.

In [ ]:
def get_finger_angle(p1, p2, p3):
    # p1, p2, p3는 MediaPipe 랜드마크 객체입니다.
    
    # 1. 랜드마크를 [x, y, z] 리스트로 변환
    # 주의: z값에 가중치를 줄 수도 있지만 여기선 그대로 씁니다.
    pos1 = [p1.x, p1.y, p1.z]
    pos2 = [p2.x, p2.y, p2.z]
    pos3 = [p3.x, p3.y, p3.z]
    
    # 2. 벡터 만들기 (중간점 pos2를 기준으로)
    # v1 = pos1 - pos2
    v1 = [pos1[0]-pos2[0], pos1[1]-pos2[1], pos1[2]-pos2[2]]
    # v2 = pos3 - pos2
    v2 = [pos3[0]-pos2[0], pos3[1]-pos2[1], pos3[2]-pos2[2]]
    
    # 3. 각도 계산
    return angle_between(v1, v2)

## 3. 실시간 각도 측정 (웹캠)

이제 웹캠을 켜고 검지 손가락의 각도를 화면에 띄워봅시다.
손가락을 폈을 때와 굽혔을 때 각도가 어떻게 변하는지 확인해보세요.

In [ ]:
import cv2 as cv
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret: break
    
    frame = cv.flip(frame, 1)
    image_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    if results.multi_hand_world_landmarks:
        # world_landmarks를 쓰는 것이 각도 계산에 더 정확합니다 (3D 공간 좌표)
        for hand_landmarks in results.multi_hand_world_landmarks:
            
            # 검지 손가락: 0번(손목), 5번(뿌리), 6번(중간), 8번(끝)
            # 각도를 잴 때 보통 손바닥(0) - 뿌리(5) - 끝(8) 혹은
            # 뿌리(5) - 중간(6) - 끝(8) 등 다양하게 잡을 수 있습니다.
            # 여기서는 Hell Hand 코드 방식인 0-5-8 (손 전체 굽힘)을 흉내내보거나,
            # 더 직관적인 5-6-8 (관절 꺾임)을 해봅시다.
            
            # 테스트: 0(손목) - 5(검지뿌리) - 8(검지끝)
            # 손가락이 펴지면 일직선(180도), 굽혀지면 각도가 줄어듦
            p_wrist = hand_landmarks.landmark[0]
            p_root = hand_landmarks.landmark[5]
            p_tip = hand_landmarks.landmark[8]
            
            angle = get_finger_angle(p_wrist, p_root, p_tip)
            
            # 화면에 표시 (landmarks는 그림 그리기용 2D 좌표를 써야 함)
            # ...하지만 간단히 텍스트만 띄웁시다.
            cv2.putText(frame, f"Index Angle: {int(angle)}", (10, 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

## 4. 마무리 및 과제

검지 손가락의 각도가 잘 나오나요?
손을 쫙 폈을 때와 주먹을 쥐었을 때의 각도를 기록해두세요.
내일 배울 **모터 매핑** 시간의 핵심 데이터가 됩니다.

**[과제]** 엄지, 중지, 약지, 소지의 각도도 각각 구해서 화면에 띄워보세요.